## 설명

1. 크롤링을 통해 나무위키
2. github의 오픈소스 활용
3. 인터넷에서 검색하여 다운로드 받은 자료 활용

위 자료들을 활용하여 my_dictionary.txt 에 욕 또는 비속어 리스트를 저장

In [1]:
# !pip3 install selenium
# !pip3 install webdriver-manager

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--lang=ko_KR')
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument('--window-size=1920x1080')

    # headless임을 숨기기 위해서
    # headless인 경우 Cloudflare 서비스가 동작한다.
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

    driver = webdriver.Chrome(service=Service(
        ChromeDriverManager().install()), options=chrome_options)
    return driver

In [2]:
def get_html() :
    driver = set_chrome_driver()

    url = "https://namu.wiki/w/%EC%9A%95%EC%84%A4/%ED%95%9C%EA%B5%AD%EC%96%B4"
    driver.get(url=url)
    html = driver.page_source
    driver.close()
    
    return html

In [3]:
html = get_html()



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_mac64.zip
Driver has been saved in cache [/Users/magyeongmi/.wdm/drivers/chromedriver/mac64/102.0.5005.61]


In [4]:
from bs4 import BeautifulSoup
import re

soup = BeautifulSoup(html)
explicit = []
p = re.compile('\w*[:]')

def switch_type(text):
    # 정의 제거
    if ":" in text:
        index = text.find(':')
        text = text[:index]
    
    # 괄호 안 텍스트 제거
    if "(" in text:
        index = text.find('(')
        text = text[:index]
    
    explicit = []
    # ㄱ/ㄴ/ㄷ
    if re.compile('\w*[/]').match(text):
        temp = text.split('/')
        explicit += temp
    
    # ㄱㄴ, ㄷㄹ
    elif re.compile('\w*[,]').match(text):
        temp = text.replace(' ','').split(',')
        explicit += temp
    
    elif re.compile('[~]').match(text):
        temp = text.replace('~','')
        explicit += [temp]
    
    # 기본
    else:
        explicit += [text]
    return explicit


def find_except(l):
    my_list = []
    if "'닭/새/돌/빡'+'-대가리'" in l:
        l.remove("'닭/새/돌/빡'+'-대가리'")
        my_list += ["닭대가리", "새대가리", "돌대가리", "빡대가리"]
    if  '허접쓰레기 또는 허섭스레기' in l:
        l.remove('허접쓰레기 또는 허섭스레기')
        my_list += ["허접쓰레기"]
    if '한남충/한녀' in l:
        l.remove('한남충/한녀')
        my_list += ['한남충','한녀']
    if  "설명충  '충'중에서는 가장 수위가 낮지만 미친,제기랄,보다는 높다." in l:
        l.remove("설명충  '충'중에서는 가장 수위가 낮지만 미친,제기랄,보다는 높다.")
        my_list += ['설명충','미친','제기랄']
    for value in l:
        if len(value) != 1:
            my_list.append(value)
    return my_list
    

for ul in soup('ul'):
    if('class' in ul.attrs and 'U9YMDCuS' in ul['class']):
        div = ul.find("div")
        explicit += switch_type(div.get_text())


In [5]:
# 예외처리
explicit = find_except(explicit)

explicit = set(explicit)

In [6]:
# github에서 불러오기

import json, requests

url = 'https://raw.githubusercontent.com/organization/Gentleman/master/resources/badwords.json'

resp = requests.get(url)
resp_parsed = re.sub(r'^jsonp\d+\(|\)\s+$', '', resp.text)
data = json.loads(resp_parsed)
github_explicit = data['badwords']

#def find_github_except(l):


explicit.update(github_explicit)

In [7]:
# https://jizard.tistory.com/288#google_vignette 에서 다운받은 비속어 리스트

f = open("./download/fword_list.txt", 'r')
file_explicit = []
while True:
    line = f.readline()
    if not line: 
        break
    file_explicit.append(line[:-1])
f.close()

explicit.update(file_explicit)

In [8]:
import docx

document = docx.Document()

templ = docx.Document("./download/badwords.docx")
document_explicit = []
for line, p in enumerate(templ.paragraphs):
    document_explicit.append(p.text.replace(' ',''))

explicit.update(document_explicit)

In [9]:
explicit.add("ㅈㄴ")

print(explicit)

{'꼬츄', '', '땡추', '쉬밸년', 'ㅁl친', '씨벌쉐이', '과메기', '병닥', '흐접', '부랄.', '찐따', '시발새끼', '18새끼', 'ㄲㅏ', 'penis', '좆까', '뒈진', '개수작', '왜놈', '아헤가오', '샹뇬', '미칀', 'ㅆI발', '색갸', '씨방놈', '식키', '눈깔', 'se', '새애액스', '섹히', '씹탱굴이', '젖탱이', '육시랄 ', '샷업', 'X끼', '자쥐', '씨발년', '쌕쓰', '강간', '후레자식애', 'ㅅㅐ끼', '시이펄', '염뵹', '18년', '띠방', '옌장', '염병할', '육갑', '접년', '창뇬', '주둥이', '쥬디', '쟘지', 'ㄲㅑ', '번섹', '돼지새끼', '쓉창', '유방', '늬기미', '자쥣', '니미씹', '씨블년', '붕알', 'ㅈㄹ', '좃도', '죠또', '워마드', '호구', '쉐끼', '실프', '존만한', '쓰이불', '개쉽', '유방핧어', 'asshole', '빙쉰', '쓰브랄쉽세', '시펄', 'ㅅHㄲI', '박쥐', "새'꺄", '귀두', '젓밥', '쌍넘', 'cunt', '10세리', '띠펄', '냄비', '싸가지가 없다, 싸가지', '똘추', '개새', '씹뇬', '쓰바새끼', '상폐녀', '저빠지게', '조랭', '씁새', '색끼', '점물', '유방쪼물럭', '나쁜 새끼', '젓가튼', '빠수니', '떡치기', '허버리년', 'ㅁ1친년', '개불랄', '어미따먹자', '좃나', '니거', '따식', '막대쑤셔줘', '미췬넘', '급식', '창녀', '개지랄', '촛불좀비', '꽂추', '게자식', '욧나', '니애미', '왕털자지', '좆털', '가슴주물럭', '10쎄', '쒸바', '붕가', '개잡년', '믜친', '18스', 'ㅃr큐', '씨불', '바보멍청이해삼멍게말미잘', '10알', '씸새끼', '버지빨아', '왕따', '18세끼', '꺼져', '새뤼', '좆같은놈', '이새끼', '스팽', '븅', '창년', '좇나'

In [10]:
explicit = list(explicit)
f = open("my_dictionary.txt",'w')
for i in explicit:
    if i != '':
        f.write(f'{i}\tNNP \n')
f.close()